In [11]:
import pandas as pd

In [13]:
df_country_extended = pd.read_csv('./../data/countries_extended.csv')
df_weather = pd.read_parquet('./../data/weather_compressed_new.parquet')
df_aggregated_average_precipitation = pd.read_csv('./../data/country_overall/average_precipitation_per_day_global.csv')
df_aggregated_average_temp = pd.read_csv('./../data/country_overall/average_temp_per_day_global.csv')


In [34]:
for row in df_country_extended[['country', 'continent']].itertuples(index=False):
    country, continent = row
    df_intermediate = df_weather[df_weather['country'] == country][['date', 'avg_temp_c', 'min_temp_c', 'max_temp_c', 'precipitation_mm']]
    df_intermediate['avg_temp_c'] = df_intermediate['avg_temp_c'].interpolate(method='linear')
    df_intermediate['min_temp_c'] = df_intermediate['min_temp_c'].interpolate(method='linear')
    df_intermediate['max_temp_c'] = df_intermediate['max_temp_c'].interpolate(method='linear')
    df_intermediate['precipitation_mm'] = df_intermediate['precipitation_mm'].interpolate(method='linear')


    precipitation = df_aggregated_average_precipitation[df_aggregated_average_precipitation['continent'] == continent][['precipitation_mm', 'date']]
    avg_temp_c = df_aggregated_average_temp[df_aggregated_average_temp['continent'] == continent][['avg_temp_c', 'date']]
    df_intermediate = df_intermediate.merge(avg_temp_c, on='date')
    df_intermediate = df_intermediate.merge(precipitation, on='date')

    df_intermediate.rename(columns={'precipitation_mm_x': 'precipitation_mm', 'avg_temp_c_y': 'avg_temp_c_continent', 'precipitation_mm_y': 'precipitation_mm_continent', 'avg_temp_c_x': 'avg_temp_c'}, inplace=True)


    global_map_compitable_name = df_country_extended.loc[df_country_extended['country'] == country][['NAME']].values[0][0]

    df_intermediate.to_csv('./../data/country_detail/' + global_map_compitable_name + '.csv', index=False)